In [1]:
# All imports will be here:
import pandas as pd
import numpy as np
from utils import import_and_transform
from utils import evaluate_model
from utils import aggregate
from utils import evaluate_model_log_reg
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (classification_report, confusion_matrix, 
                             roc_auc_score, roc_curve, precision_recall_curve)
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


In [2]:

def import_and_transform(data):
    """Import and basic preprocessing only."""
    if isinstance(data, str):
        df = pd.read_parquet(data)
    else:
        df = data
    
    df = df[df['userId'] != '']
    df['userId'] = df['userId'].astype(int)
    df["gender"] = df["gender"].map({'F': 0, 'M': 1})
    df["level"] = df["level"].map({'free': 0, 'paid': 1})
    df['ts'] = pd.to_datetime(df['ts'], unit='ms')
    df['registration'] = pd.to_datetime(df['registration'])
    
    df['session_length'] = df.groupby(['userId', 'sessionId'])['ts'].transform(
        lambda x: (x.max() - x.min()).total_seconds()
    )
    df['song_played'] = (df['page'] == 'NextSong').astype(int)
    
    return df


def aggregate_features(data, observation_end):
    """Calculate features using only data up to observation_end."""
    observation_end = pd.Timestamp(observation_end)
    
    user_df = data.groupby('userId').agg({
        'gender': 'first',
        'registration': 'first',
        'level': lambda x: x.mode().iloc[0] if not x.mode().empty else 0,
        'sessionId': 'nunique',
        'itemInSession': 'max',
        'ts': ['min', 'max'],
        'session_length': 'mean',
        'song_played': 'sum',
        'artist': pd.Series.nunique,
        'length': 'sum'
    }).reset_index()
    
    user_df.columns = ['userId', 'gender', 'registration', 'level',
                       'num_sessions', 'max_item_in_session', 'ts_min', 'ts_max', 
                       'avg_session_length_seconds', 'num_songs_played', 
                       'unique_artists', 'total_length']
    
    user_df['days_active'] = (observation_end - user_df['ts_min']).dt.days
    user_df['membership_length'] = (observation_end - user_df['registration']).dt.days
    user_df['days_since_last_activity'] = (observation_end - user_df['ts_max']).dt.days
    user_df['songs_per_day'] = user_df['num_songs_played'] / (user_df['days_active'] + 1)
    user_df['sessions_per_day'] = user_df['num_sessions'] / (user_df['days_active'] + 1)
    
    user_df = user_df.fillna(0)
    user_df.set_index('userId', inplace=True)
    
    return user_df


def get_churned_users(df, start_date, end_date):
    """Get users who churned between start_date and end_date."""
    start = pd.Timestamp(start_date)
    end = pd.Timestamp(end_date)
    
    cancellations = df[df['page'] == 'Cancellation Confirmation']
    churned = cancellations[(cancellations['ts'] > start) & 
                           (cancellations['ts'] <= end)]['userId'].unique()
    return set(churned)


# Load training data
df_train = import_and_transform('Data/train.parquet')

# Prepare test data
df_test = import_and_transform('Data/test.parquet')



In [3]:
# Create multiple training samples with sliding window
training_dates = pd.date_range('2018-10-15', '2018-11-05', freq='5D')

X_train_list = []
y_train_list = []

for obs_date in training_dates:
    # Features from data up to obs_date
    df_obs = df_train[df_train['ts'] <= obs_date]
    features = aggregate_features(df_obs, obs_date)
    
    # Labels from next 10 days
    window_end = obs_date + pd.Timedelta(days=10)
    churned_users = get_churned_users(df_train, obs_date, window_end)
    
    # Convert to Series with same index as features
    labels = pd.Series(
        features.index.isin(churned_users).astype(int),
        index=features.index,
        name='churned'
    )
    
    X_train_list.append(features)
    y_train_list.append(labels)
    
    print(f"Obs date: {obs_date.date()}, Users: {len(features)}, Churn rate: {labels.mean():.2%}")

# Combine all training samples
X_train_combined = pd.concat(X_train_list)
y_train_combined = pd.concat(y_train_list)

# Drop non-feature columns
feature_cols = X_train_combined.select_dtypes(include=[np.number]).columns
feature_cols = [c for c in feature_cols if c not in ['registration', 'ts_min', 'ts_max', 'total_length']]
X_train_final = X_train_combined[feature_cols]

test_features = aggregate_features(df_test, '2018-11-20')
X_test = test_features[feature_cols]


Obs date: 2018-10-15, Users: 16271, Churn rate: 5.08%
Obs date: 2018-10-20, Users: 17347, Churn rate: 4.48%
Obs date: 2018-10-25, Users: 17888, Churn rate: 4.49%
Obs date: 2018-10-30, Users: 18271, Churn rate: 4.46%
Obs date: 2018-11-04, Users: 18592, Churn rate: 3.78%


In [6]:
!pip install torch scikit-learn

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

# Calculate scale_pos_weight
scale_pos_weight = (y_train_combined == 0).sum() / (y_train_combined == 1).sum()
print(f"Scale pos weight: {scale_pos_weight:.2f}")

# Split for validation
X_train_nn, X_val_nn, y_train_nn, y_val_nn = train_test_split(
    X_train_final, y_train_combined, test_size=0.2, random_state=42, stratify=y_train_combined
)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_nn)
y_train_tensor = torch.FloatTensor(y_train_nn.values).reshape(-1, 1)
X_val_tensor = torch.FloatTensor(X_val_nn)
y_val_tensor = torch.FloatTensor(y_val_nn.values).reshape(-1, 1)

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)

# Define Neural Network (same architecture as your TensorFlow version)
class ChurnNN(nn.Module):
    def __init__(self, input_size, hidden_units=128, dropout_rate=0.3):
        super(ChurnNN, self).__init__()
        
        self.layer1 = nn.Linear(input_size, hidden_units)
        self.bn1 = nn.BatchNorm1d(hidden_units)
        self.dropout1 = nn.Dropout(dropout_rate)
        
        self.layer2 = nn.Linear(hidden_units, hidden_units // 2)
        self.bn2 = nn.BatchNorm1d(hidden_units // 2)
        self.dropout2 = nn.Dropout(dropout_rate)
        
        self.layer3 = nn.Linear(hidden_units // 2, hidden_units // 4)
        self.dropout3 = nn.Dropout(dropout_rate / 2)
        
        self.output = nn.Linear(hidden_units // 4, 1)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        
        x = self.layer2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        
        x = self.layer3(x)
        x = self.relu(x)
        x = self.dropout3(x)
        
        x = self.output(x)
        x = self.sigmoid(x)
        
        return x

# Create model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = ChurnNN(input_size=X_train_final.shape[1], hidden_units=128, dropout_rate=0.3).to(device)

# Calculate class weights
pos_weight = torch.FloatTensor([scale_pos_weight]).to(device)

# Loss and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_epoch(model, loader, criterion, optimizer, device, pos_weight):
    model.train()
    total_loss = 0
    
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        # Apply class weights manually
        optimizer.zero_grad()
        outputs = model(X_batch)
        
        # Weight the loss for positive class
        weights = torch.where(y_batch == 1, pos_weight, torch.ones_like(y_batch))
        loss = (criterion(outputs, y_batch) * weights.squeeze()).mean()
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)

# Validation function
def validate(model, loader, device):
    model.eval()
    predictions = []
    actuals = []
    
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            predictions.extend(outputs.cpu().numpy())
            actuals.extend(y_batch.numpy())
    
    from sklearn.metrics import roc_auc_score
    auc = roc_auc_score(actuals, predictions)
    return auc

# Training loop
print("\n🔍 Training Neural Network...")
best_auc = 0
patience = 20
patience_counter = 0

for epoch in range(100):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device, pos_weight)
    val_auc = validate(model, val_loader, device)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/100 | Loss: {train_loss:.4f} | Val AUC: {val_auc:.4f}")
    
    # Early stopping
    if val_auc > best_auc:
        best_auc = val_auc
        patience_counter = 0
        torch.save(model.state_dict(), 'best_nn_model.pth')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch+1}")
            break

print(f"Best validation AUC: {best_auc:.4f}")

# Load best model
model.load_state_dict(torch.load('best_nn_model.pth'))

# ============================================================================
# CREATE WRAPPER FOR evaluate_model
# ============================================================================

class PyTorchModelWrapper:
    """Wrapper to make PyTorch model compatible with evaluate_model"""
    
    def __init__(self, pytorch_model, device):
        self.model = pytorch_model
        self.device = device
        self.model.eval()
    
    def predict_proba(self, X):
        """Return probabilities in sklearn format"""
        with torch.no_grad():
            X_tensor = torch.FloatTensor(X).to(self.device)
            proba_class_1 = self.model(X_tensor).cpu().numpy().flatten()
            proba_class_0 = 1 - proba_class_1
            return np.column_stack([proba_class_0, proba_class_1])
    
    def predict(self, X):
        """Return binary predictions"""
        proba = self.predict_proba(X)
        return (proba[:, 1] >= 0.5).astype(int)

# Wrap the model
wrapped_nn_model = PyTorchModelWrapper(model, device)

# Use evaluate_model
print("\n📊 Evaluating Neural Network...")
evaluate_model(wrapped_nn_model, X_test, 0.5, file_out='nn_model.csv')

# ============================================================================
# BONUS: Find optimal threshold
# ============================================================================

from sklearn.metrics import precision_recall_curve

# Get validation predictions
with torch.no_grad():
    X_val_tensor = X_val_tensor.to(device)
    y_proba_val = model(X_val_tensor).cpu().numpy().flatten()

# Find optimal threshold
precisions, recalls, thresholds = precision_recall_curve(y_val_nn, y_proba_val)
f1_scores = 2 * (precisions[:-1] * recalls[:-1]) / (precisions[:-1] + recalls[:-1] + 1e-10)
optimal_threshold = thresholds[np.argmax(f1_scores)]

print(f"\n🎯 Optimal threshold: {optimal_threshold:.4f}")

# Evaluate with optimal threshold
evaluate_model(wrapped_nn_model, X_test, optimal_threshold, file_out='nn_model_optimal.csv')

print("\n✅ Neural Network Complete!")
print(f"Created: nn_model.csv (threshold=0.5)")
print(f"Created: nn_model_optimal.csv (threshold={optimal_threshold:.4f})")

  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.2/110.9 MB 3.5 MB/s eta 0:00:32
   ---------------------------------------- 0.8/110.9 MB 8.5 MB/s eta 0:00:13
    --------------------------------------- 1.5/110.9 MB 11.6 MB/s eta 0:00:10
    --------------------------------------- 2.0/110.9 MB 11.8 MB/s eta 0:00:10
   - -------------------------------------- 2.8/110.9 MB 12.1 MB/s eta 0:00:09
   - -------------------------------------- 3.4/110.9 MB 12.1 MB/s eta 0:00:09
   - -------------------------------------- 4.1/110.9 MB 12.5 MB/s eta 0:00:09
   - -------------------------------------- 4.8/110.9 MB 12.9 MB/s eta 0:00:09
   - -------------------------------------- 5.4/110.9 MB 13.3 MB/s eta 0:00:08
   -- ------------------------------------- 5.8/110.9 MB 12.7 MB/s eta 0:00:09
   -- ------------------------------------- 6.6/110.9 MB 13.1 MB/s eta 0:00:08


OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "c:\Users\andre\anaconda3\Lib\site-packages\torch\lib\c10.dll" or one of its dependencies.